In [1]:
from functions import *
from torch.optim import Adam

In [2]:
max_ = -1
for k,v in mapping.items():
    if v>max_:
        k_max = k
        max_ = v

In [4]:
evaluate(matrix[:,np.array(k_max)],np.array(k_max),policy_lookup,len(k_max))

2.1029411764705874

In [4]:
max_

2.1029411764705874

In [2]:
from functions_pt import *

In [3]:
embed_size = 128
dropout = 0.1
multiple_factor = 1
layers = 2
batch_size = 64
agg = 'mean' # mean,sum,min,max
model = pointNet(layers,embed_size,agg,dropout,multiple_factor)#.to('cuda')
model.load_state_dict(torch.load('baseline0.pt'))
model.eval()
model = model.to('cuda')

In [4]:
words_embed = np.load('possible_words_embed.npy')

In [8]:
# slightly worse than # of steps required by mapping as expected
evaluate(matrix[:,np.array(k_max)],np.array(k_max),policy_model,len(k_max),model=model,words_embed=words_embed,top=0.6)

2.1249999999999987

In [5]:
# see how well model does on full problem that it is not trained on
#     if count == 1: return 1
#     if count == 2: return 1.5
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.5036812472932013

RL Loop

In [ ]:
lr = 6e-5
clip = 1
epochs = 4
damp = 2
iterations = 15
max_data = 0

for i in range(iterations):
    # policy evaluation
    #model.eval()
    score = evaluate_save(matrix,np.arange(2309),policy_model,2309,0,model=model,words_embed=words_embed,top=0.6)
    print('iter:{}, score:{}'.format(i,score))
    
    # prepare data for training
    index,val,log_prob = zip(*out)
    prob = np.array(log_prob)
    prob -= prob.mean()
    prob = np.exp(prob/damp)
    prob = torch.tensor(prob,dtype=torch.float32,device='cuda')
    length = torch.tensor([len(i) for i in index],dtype=torch.float32,device='cuda')
    words = torch.tensor(words_embed[np.concatenate(index)]).long().to('cuda')
    ys = torch.tensor(val,dtype=torch.float32,device='cuda')

    
    # policy update
    #model.train()
    opt = Adam([    {'params': [p for p in model.parameters() if p is not model.w]},\
                {'params': model.w, 'lr': lr/4}
            ],lr=lr)
    for _ in range(epochs):
        yhat = model((words,length))
        loss = torch.mean(prob*(ys-yhat)**2)
        loss.backward()
        clip_grad_value_(model.parameters(),clip)
        opt.step()
        opt.zero_grad()
    #print(model.w)
    while len(out)>max_data:
        out.pop(0)

iter:0, score:3.5036812472932013
iter:1, score:3.50238198354266
iter:2, score:3.5032481593763545
iter:3, score:3.5010827197921186
iter:4, score:3.501948895625813
iter:5, score:3.518406236466002
iter:6, score:3.498484192291036
